In [6]:
labels=['your', 'ready', 'safe', 'order', 'fork', 'repeat', 'sandwich', 'biscuit', 'please', 'wait', 'sugar', 'cup', 'sweet', 'ac', 'refill', 'lettuce', 'bill', 'thank-you', 'water', 'no', 'vegetables', 'dessert', 'pop corn', 'would', 'bitter', 'milk', 'enjoy', 'cash', 'allergy', 'pepsi', 'coca-cola', 'is', 'spicy', 'nacho', 'cost', 'fresh', 'ice cream', 'chair', 'cheese', 'napkin', 'good night', 'mayonnaise', 'about', 'brownie', 'soda', 'bye', 'coupon', 'small', 'mustard', 'urgent', 'credit card', 'french fries', 'ketchup', 'you_ll', 'manager', 'salt', 'eat', 'total', 'receipt', 'juicy', 'cold', 'extra', 'lid', 'yogurt', 'warm', 'menu', 'ingredients', 'burger', 'alcohol', 'relish', 'eggs', 'chicken', 'hot', 'what', 'tissues', 'pickle', 'spoon', 'bread', 'additional', 'hello', 'straw', 'sauce', 'lactose', 'bag', 'drive', 'pizza', 'bacon', 'pepper', 'tomato', 'sorry', 'coke', 'drink', 'which', 'gluten free', 'I_ve', 'cake', 'barbecue']

In [7]:
max_index = np.argmax(pred)
print(labels[max_index])

bread


In [13]:
import os
import cv2
import numpy as np
from keras.models import load_model
pred_labels=[]
# Load pre-trained LRCN model
model = load_model('/Users/vedantpadole/Desktop/Research/LRCN_finalized.h5')

# Set parameters for sequence length and image size
seq_length = 10
img_height = 128
img_width = 128

# Initialize buffer for video frames
frame_buffer = []

# Initialize list for storing predicted labels
label_list = []

# Create video capture object
cap = cv2.VideoCapture(0)

# Define codec and output path for saving video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_folder = 'output_videos'
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, 'output.mp4')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (640, 480))

# Set timer for label prediction
timer = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Resize frame to desired height and width
    frame_resized = cv2.resize(frame, (img_height, img_width))
    
    # Add frame to buffer
    frame_buffer.append(frame_resized)
    
    # Once buffer is filled, predict on sequence of frames
    if len(frame_buffer) == seq_length:
        # Convert buffer to numpy array
        input_seq = np.array(frame_buffer)
        
        # Preprocess input sequence
        input_seq = input_seq.astype('float32') / 255.0
        
        # Reshape input sequence for LRCN model
        input_seq = np.expand_dims(input_seq, axis=0)
        
        # Make prediction on input sequence
        pred = model.predict(input_seq)
        pred_label = np.argmax(pred)
        
        # Add predicted label to list
        label_list.append(pred_label)
        
        # Clear buffer
        frame_buffer = []
    
    # Display original frame
    cv2.imshow('Live video', frame)
    
    # Write frame to output video file
    out.write(frame)
    
    # Increment timer
    timer += 1
    
    # If timer reaches 60 (i.e. 3 seconds have elapsed), predict label
    if timer == 60:
        # If there are at least 3 labels in the list, compute mode of last 3 labels
        if len(label_list) >= 2.5:
            mode_label = np.argmax(np.bincount(label_list[-3:]))
            pred_labels.append(mode_label)
            print('Predicted label:', mode_label)
        # Reset timer and label list
        timer = 0
        label_list = []
    
    # Exit loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture object, output file, and close all windows
cap.release()
out.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 38ms/step
Predicted label: 13
1/1 [==============================] - 0s 36ms/step
Predicted label: 60
1/1 [==============================] - 0s 39ms/step
Predicted label: 60
1/1 [==============================] - 0s 36ms/step
Predicted label: 60
1/1 [==============================] - 0s 39ms/step
Predicted label: 60
1/1 [==============================] - 0s 38ms/step
Predicted label: 60
1/1 [==============================] - 0s 38ms/step


In [14]:
final=[]
for i in pred_labels:
    final.append(labels[i-1])
final

['sweet', 'juicy', 'juicy', 'juicy', 'juicy', 'juicy']

In [15]:
final=set(final)
final

{'juicy', 'sweet'}